In [17]:
import os
import time

import dill
import matplotlib.pyplot as plt
import pandas as pd

from collab.foraging import central_park_birds as cpb
from collab.foraging import toolkit as ft
from collab.utils import find_repo_root

root = find_repo_root()

# the smoke test code is to make sure that the code runs without error
# during continuous integration
# can be ignored by the reader
smoke_test = "CI" in os.environ
frames = 150 if smoke_test else 300
frames_sps = 150 if smoke_test else 2300
sampling_rate = 0.001 if smoke_test else 0.01

notebook_starts = time.time()